## Import Libraries

In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz
from time import time

## Load the data

In [2]:
df_correct = pd.read_csv('Correct_cities.csv')
df_misspelt = pd.read_csv('Misspelt_cities.csv')
print(df_correct.shape)
print(df_misspelt.shape)

(23018, 3)
(23018, 2)


## EDA

In [3]:
print(df_correct.head())
print(df_misspelt.head())

               name               country       id
0      les Escaldes               Andorra  3040051
1  Andorra la Vella               Andorra  3041563
2    Umm al Qaywayn  United Arab Emirates   290594
3    Ras al-Khaimah  United Arab Emirates   291074
4      Khawr Fakkān  United Arab Emirates   291696
    misspelt_name        country
0  Hfjdúszoposzló        Hungary
1        Otrajnyy         Russia
2      ian Isidre           Peru
3   Bordj Zemoufa        Algeria
4     ChulamViwta  United States


In [13]:
print(df_correct.dropna().shape)
print(df_misspelt.dropna().shape)

(23018, 3)
(23018, 2)


## Map the incorrect cities to correct cities using fuzzy matching

As the record count is high, the mapping process may take around 45 mins based on the system's processing power

Comment the lines 11 and 12 to see the results for the whole data

In [15]:
df_misspelt['id'] = -1
start = time()
for idx, row in df_misspelt.iterrows():
    max_fuzz=0; max_fuzz_idx=None;
    for corr_idx, corr_row in df_correct[df_correct['country']==row['country']].iterrows():
        fuzz_match = fuzz.ratio(row['misspelt_name'], corr_row['name'])
        if fuzz_match > max_fuzz:
            max_fuzz = fuzz_match
            max_fuzz_idx = corr_idx
    df_misspelt.loc[df_misspelt.index==idx, 'id'] = df_correct.iloc[max_fuzz_idx]['id']
    if idx>200:
        break
end = time()
print("total time - ", end - start)

total time -  26.470224857330322


In [16]:
df_misspelt.shape

(23018, 3)

In [18]:
df_merged = pd.merge(df_correct, df_misspelt, on=['id','country'], how='inner')
df_merged

,name,country,id,misspelt_name
0,Armavir,Armenia,616631,Asmavir
1,Luanda,Angola,2240449,Luadda
2,San Jorge,Argentina,3837441,San Jmrge
3,Quitilipi,Argentina,3839490,Quitflipi
4,Earlwood,Australia,2167949,Esrlwood
...,...,...,...,...
197,Charallave,Venezuela,3645854,Charalqava
198,Vĩnh Long,Vietnam,1562693,Vĩnh Lone
199,Thành Phố Phủ Lý,Vietnam,1570449,Thành Phố fer Lý
200,Thành Phố Hạ Long,Vietnam,1580410,Thành Pnố yạaLong


In [4]:
df_misspelt.to_csv('Misspelt_cities_updated.csv')